In [ ]:
import os
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from natsort import natsorted

In [ ]:
# Set the path to the directory containing the image stacks
stack_dir = 'C:/Users/au519867/OneDrive - Aarhus universitet/Skrivebord/Raw_Images/RAW_Stacks/Threshold'

# Get a list of all TIFF files in the input directory
files = [f for f in os.listdir(stack_dir) if f.endswith('.tiff')]

green_stack = tifffile.imread(os.path.join(stack_dir, files[0]))
red_stack = tifffile.imread(os.path.join(stack_dir, files[1]))

In [ ]:

# Define number of sections for each image to calculate means
num_sections = 3

# Calculate the height of each section
section_height = green_stack.shape[1] // num_sections

# Loop over each image in the green stack and divide it into sections
green_section_means = []
green_sections = []
for image in green_stack:
    # Divide the image into sections
    sections = [image[i*section_height:(i+1)*section_height, :] for i in range(num_sections)]
    green_sections.append(sections)
    
    # Calculate the mean intensity of each section and append it to the list
    green_section_mean = [np.nanmean(section) for section in sections]
    green_section_means.append(green_section_mean)

green_section_means = np.reshape(green_section_means, (-1, num_sections))
print(green_section_means)

# Loop over each image in the red stack and divide it into sections
red_section_means = []
red_sections = []
for image in red_stack:
    sections = [image[i*section_height:(i+1)*section_height, :] for i in range(num_sections)]
    red_sections.append(sections)
    
    # Calculate the mean intensity of each section and append it to the list
    red_section_mean = [np.nanmean(section) for section in sections]
    red_section_means.append(red_section_mean)

red_section_means = np.reshape(red_section_means, (-1, num_sections))
print(red_section_means)


In [ ]:
# Define the image index to show
image_idx = 0

# Get an example image from the green stack and its sections to see how its divided
image = green_stack[image_idx]
sections = green_sections[image_idx]

for section_idx in range(num_sections):
    section = sections[section_idx]
    plt.imshow(section, cmap="gray")
    plt.title(f'Image {image_idx}, Section {section_idx}')
    plt.show()


In [ ]:
green_mean_of_means = []
for i in range(len(green_section_means)):
    mean_green = np.nanmean(green_section_means[i])
    green_mean_of_means.append(mean_green)

print(green_mean_of_means)

red_mean_of_means = []
for i in range(len(red_section_means)):
    mean_red = np.nanmean(red_section_means[i])
    red_mean_of_means.append(mean_red)

print(red_mean_of_means)


In [ ]:
# Loop through all pairs of images in the directory
for i in range(len(green_stack)):

    # Calculate the intensity ratio between the two images
    ratio = red_mean_of_means[i] / green_mean_of_means[i]
    
    # Print the intensity ratio
    print(f'Image {i+1}: Intensity ratio = {ratio:.2f}')

In [ ]:
# sort filenames to match with stack index
files = natsorted(files)
# Initialize an empty dataframe
df_ratios = pd.DataFrame(columns=['Image', 'Intensity Ratio'])

# Loop through all pairs of images in the directory
for i in range(len(green_stack)):
    # Calculate the intensity ratio between the two images
    ratio = red_mean_of_means[i] / green_mean_of_means[i]
    
    # Add the ratio to the dataframe, along with the filename of the green image
    filename = os.path.splitext(files[i])[0]  # get the filename without extension
    df_ratios.loc[i] = [filename, ratio]

# Round the intensity ratios to 2 decimal points
df_ratios['Intensity Ratio'] = df_ratios['Intensity Ratio'].round(2)

# Print the dataframe
print(df_ratios)